In [22]:
from IPython.display import display, Markdown, HTML
import math

#  Mathematik im Galois-Feld GF(2^8)

print("Galois Feld GF(2^8)")
print("Alle Berechnungen müssen Wert zwischen 0-255 zurückgeben!")

# Init log und anti-log für lookup
exp_table = [0] * 256
log_table = [0] * 256
primitive_polynomial = 0b100011101 # = 0x11D = x^8 + x^4 + x^3 + x^2 + 1

# Tabellen erstellen
x = 1
for i in range(255):
    exp_table[i] = x
    log_table[x] = i
    x <<= 1
    if x & 0x100:
        x ^= primitive_polynomial

exp_table[255] = exp_table[0]

# a = 45, b = 173 => a * b in GF(2^8):
# log_a = log_tabe[a], log_b = log_table[b]
# log_result = (log_a + log_b) % 255
# result = exp_table[log_result]

# Multiplikation innerhalb des GF(2^8)-Feld
def gf_multiply(a, b):
    if a == 0 or b == 0:
        return 0
    return exp_table[(log_table[a] + log_table[b]) % 255]

# Polynom-Multiplikation im GF(2^8)-Feld
def gf_poly_multiply(p1, p2):
    result_len = len(p1) + len(p2) - 1
    result = [0] * result_len
    for i in range(len(p1)):
        for j in range(len(p2)):
            result[i + j] ^= gf_multiply(p1[i], p2[j])
    return result

# Polynomdivision
def gf_poly_div_detailed(dividend, divisor, printb = False):
    result = list(dividend)
    divisor_len = len(divisor)
    if printb:
      display(Markdown(f"**Dividend:** `{result}`\n\n**Divisor:** `{divisor}`"))

    for i in range(len(dividend) - divisor_len + 1):
        coefficient = result[i]

        if printb:
          display(Markdown(f"--- \n### PolyDiv: Schritt {i+1}"))

        if coefficient == 0:
            if printb:
              display(Markdown("Führende Ziffer ist 0, kein Rechenschritt nötig. Wir rücken eins weiter."))
            continue

        term_to_subtract = [0] * i + [gf_multiply(d, coefficient) for d in divisor]
        term_to_subtract.extend([0] * (len(result) - len(term_to_subtract)))

        div_str = "&nbsp;".join(f"{n:03}" for n in result)
        sub_str = "&nbsp;".join(f"{n:03}" for n in term_to_subtract)

        if printb:
          display(Markdown(f"Die vorderste Ziffer ist **{coefficient}**. Wir müssen das **{coefficient}-fache des Divisors** subtrahieren (XOR)."))

        next_result = [a ^ b for a, b in zip(result, term_to_subtract)]
        res_str = "&nbsp;".join(f"{n:03}" for n in next_result)

        html_calculation = f"""
        <div style="font-family: monospace; line-height: 1.6; font-size: 14px;">
          &nbsp;&nbsp;&nbsp;&nbsp;{div_str} (Aktueller Dividend)<br/>
          XOR {sub_str} ({coefficient} * Divisor)<br/>
          ------------------------------------------------------------------<br/>
          = &nbsp;&nbsp;{res_str} (Neuer Zwischenstand)
        </div>
        """
        if printb:
          display(HTML(html_calculation))

        result = next_result

    remainder = result[-(divisor_len - 1):]
    return remainder

# Format: (Total Data Codewords, Total Error Correction Codewords)
QR_CAPACITY_TABLE_V1 = {
    'L': (19, 7),
    'M': (16, 10),
    'Q': (13, 13),
    'H': (9, 17)
}
CHOSEN_LEVEL = 'Q' # Medium gewählt
DATA_CAPACITY, num_error_bytes = QR_CAPACITY_TABLE_V1[CHOSEN_LEVEL]

print("\nFolgendes Level gewählt: ", CHOSEN_LEVEL)
print(f"Anzahl Daten-Bytes: {DATA_CAPACITY}")
print("Anzahl Fehlerkorrektur-Bytes:", num_error_bytes)

# Schritt 1: Daten in Bytes umwandeln

###################################################
input_string = "ELDAR"
###################################################

message_bytes = [ord(c) for c in input_string]
print("\nUmwandeln von Nachricht in Bytes")
print(f"Nachricht: `{input_string}`")
print(f"Bytes: `{message_bytes}`")
# Schritt 1.5: Füll-Bytes
print("Rest des Platzes mit Füll-Bytes auffüllen (abwechselnd 236, 17)")

padding_bytes = []
padding_needed = DATA_CAPACITY - len(message_bytes)
for i in range(padding_needed):
    if i % 2 == 0:
        padding_bytes.append(236) # Standard-Padding-Byte 1
    else:
        padding_bytes.append(17)  # Standard-Padding-Byte 2

data_payload = message_bytes + padding_bytes

print(f"Voller Datenblock (Nachricht + Padding)", data_payload)

# Schritt 2: Das standardisierte Generatorpolynom
print("\nGeneratorpolynom erstellen (nach QR-Code-Standard):")

# Generatorpolynom (g(x)), für t Fehlerkorrektur-Symbole:
# g(x) = (x - α^0)(x - α^1)...(x - α^(t-1))

generator_poly = [1]
for i in range(num_error_bytes):
    generator_poly = gf_poly_multiply(generator_poly, [1, exp_table[i]])

print(f"`{generator_poly}`")

# Schritt 3: Nachrichtenpolynom für die Division vorbereiten
print(f"\nWir hängen nun {num_error_bytes} Nullen an unseren Datenblock an, um Platz für den Rest zu schaffen")
padded_message_bytes = data_payload + [0] * num_error_bytes

print(f"Vorbereitete Nachricht: ", padded_message_bytes)

# Schritt 4: Die Polynomdivision durchführen
print("\nJetzt teilen wir den vorbereiteten, vollen Datenblock durch das Generatorpolynom.")
error_correction_bytes = gf_poly_div_detailed(padded_message_bytes, generator_poly)

# Schritt 5: Ergebnis
print(f"Der **Rest** der Division ist: ", error_correction_bytes)

final_codeword = data_payload + error_correction_bytes

print("\nDer volle Datenblock und die Fehlerkorrektur-Bytes werden zusammengefügt.")
print(f"Finale Codewörter: ", final_codeword)


# --- Teil 3: Visualisierung der Daten (Vereinfacht) ---
print("\nNun erstellen wir eine vereinfachte Visualisierung (wie ein QR-Code) mit diesen Daten:")

# Schritt 7: Bytes in einen Bit-String umwandeln
bit_string = "".join(f"{byte:08b}" for byte in final_codeword)

print(f"Die {len(final_codeword)} Bytes werden zu einem langen String aus {len(bit_string)} Bits:")
print(f"{bit_string}")

# Add a legend
legend_html = """
<div style="margin-top: 15px; margin-bottom: 5px;">
    <b>Legende:</b>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:black; border:1px solid #ccc; margin-right:5px;"></div> Original-Daten (1)</span>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:silver; border:1px solid #ccc; margin-right:5px;"></div> Füll-Daten (1)</span>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:crimson; border:1px solid #ccc; margin-right:5px;"></div> Fehlerkorrektur-Bit (1)</span>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:white; border:1px solid #ccc; margin-right:5px;"></div> 0-Bit</span>
</div>
"""
display(HTML(legend_html))

num_message_bits = len(message_bytes) * 8
num_padding_bits = len(padding_bytes) * 8

total_bits = len(bit_string)
cols = 8 # Each row is one byte

html_grid = f"<div style='display: grid; grid-template-columns: repeat({cols}, 20px); gap: 1px; border: 2px solid #333; width: fit-content; margin-top: 10px;'>"
for i, bit in enumerate(bit_string):
    color = "white" # Default for '0' bits
    if bit == '1':
        if i < num_message_bits:
            color = "black"  # Original message
        elif i < num_message_bits + num_padding_bits:
            color = "silver" # Padding bytes
        else:
            color = "crimson" # Error correction bytes

    html_grid += f"<div style='width: 20px; height: 20px; background-color: {color};'></div>"
html_grid += "</div>"

display(HTML(html_grid))


Galois Feld GF(2^8)
Alle Berechnungen müssen Wert zwischen 0-255 zurückgeben!

Folgendes Level gewählt:  Q
Anzahl Daten-Bytes: 13
Anzahl Fehlerkorrektur-Bytes: 13

Umwandeln von Nachricht in Bytes
Nachricht: `ELDAR`
Bytes: `[69, 76, 68, 65, 82]`
Rest des Platzes mit Füll-Bytes auffüllen (abwechselnd 236, 17)
Voller Datenblock (Nachricht + Padding) [69, 76, 68, 65, 82, 236, 17, 236, 17, 236, 17, 236, 17]

Generatorpolynom erstellen (nach QR-Code-Standard):
`[1, 137, 73, 227, 17, 177, 17, 52, 13, 46, 43, 83, 132, 120]`

Wir hängen nun 13 Nullen an unseren Datenblock an, um Platz für den Rest zu schaffen
Vorbereitete Nachricht:  [69, 76, 68, 65, 82, 236, 17, 236, 17, 236, 17, 236, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Jetzt teilen wir den vorbereiteten, vollen Datenblock durch das Generatorpolynom.
Der **Rest** der Division ist:  [29, 245, 55, 210, 123, 88, 72, 194, 191, 232, 62, 160, 51]

Der volle Datenblock und die Fehlerkorrektur-Bytes werden zusammengefügt.
Finale Codewörter:  

In [23]:
# 1. Fehlererkennung: Existiert ein Fehler in den Daten?
# 2. Fehlerlokalisierung: An welchen Positionen befinden sich Fehler?
# 3. Datenkorrektur: Die Umkehrung der Fehler, um die Originaldaten wiederherzustellen

# 1. Fehlererkennung
correct = final_codeword
print("The correct codeword: ", correct)

damaged = correct.copy()

def damage(index, damagedValue):
    damaged[index] = damagedValue

damage(3, 34)
damage(7, 111)

print("The damaged codeword: ", damaged)

num_error_bytes = 0

if CHOSEN_LEVEL == 'L':
    num_error_bytes = 7
elif CHOSEN_LEVEL == 'M':
    num_error_bytes = 10
elif CHOSEN_LEVEL == 'Q':
    num_error_bytes = 13
elif CHOSEN_LEVEL == 'H':
    num_error_bytes = 17

def gf_mul(x, y, texp_table, tlog_table):
    if x == 0 or y == 0:
        return 0
    return texp_table[(tlog_table[x] + tlog_table[y]) % 255]

def poly_eval(poly, x, texp_table, tlog_table):
    result = 0
    for coeff in poly:
        result = gf_mul(result, x, texp_table, tlog_table) ^ coeff
    return result

def compute_syndromes(dmged, num_error_bytes):
    syndromes = []
    poly = list(dmged)
    for i in range(num_error_bytes):
        x = exp_table[i]
        val = poly_eval(poly, x, exp_table, log_table)
        syndromes.append(val)
    return syndromes

syndromes = compute_syndromes(damaged, num_error_bytes)
print("Syndrome:", syndromes)


The correct codeword:  [69, 76, 68, 65, 82, 236, 17, 236, 17, 236, 17, 236, 17, 29, 245, 55, 210, 123, 88, 72, 194, 191, 232, 62, 160, 51]
The damaged codeword:  [69, 76, 68, 34, 82, 236, 17, 111, 17, 236, 17, 236, 17, 29, 245, 55, 210, 123, 88, 72, 194, 191, 232, 62, 160, 51]
Syndrome: [224, 67, 190, 229, 135, 254, 123, 25, 36, 13, 254, 251, 42]


In [24]:
# 2. Fehlerlokalisierung
def gf_inverse(a):
  if a == 0: raise ZeroDivisionError()
  return exp_table[255 - log_table[a]]

def berlekamp_massey(syndromes):
    C = [1]
    B = [1]
    L = 0
    m = 1
    b = 1

    for n in range(len(syndromes)):
        d = syndromes[n]
        for i in range(1, L + 1):
            d ^= gf_multiply(C[i], syndromes[n - i])

        if d == 0:
            m += 1
        else:
            T = C[:]
            correction_factor = gf_multiply(d, gf_inverse(b))
            term = [gf_multiply(correction_factor, val) for val in B]
            term = [0] * m + term

            if len(C) < len(term):
                C.extend([0] * (len(term) - len(C)))

            for i in range(len(term)):
                C[i] ^= term[i]

            if 2 * L <= n:
                L = n + 1 - L
                B = T
                b = d
                m = 1
            else:
                m += 1

    return C[:L + 1]

# Chien-Suche zur Fehlerpositionsbestimmung
def chien_search(locator_poly, code_length, texp_table, tlog_table):
    error_positions = []
    for i in range(code_length):
        x_inv = texp_table[(255 - i) % 255]
        result = 0
        for j, coef in enumerate(locator_poly):
            result ^= gf_mul(coef, texp_table[(tlog_table[x_inv] * j) % 255], texp_table, tlog_table)
        if result == 0:
            error_positions.append(i)
    return error_positions

locator_poly_lf = berlekamp_massey(syndromes)
print("Fehler-Lokalisierer-Polynom Λ(x):", locator_poly_lf)

error_positions = chien_search(locator_poly_lf, len(damaged), exp_table, log_table)
print("Fehlerhafte Koeffizienten (Potenzen): ", error_positions)

locator_poly = list(reversed(locator_poly_lf))

error_indices = []

for position in error_positions:
    index = len(damaged) - 1 - position
    error_indices.append(index)

error_indices.sort()

print("Fehlerhafte Indizes: ", error_indices)

Fehler-Lokalisierer-Polynom Λ(x): [1, 199, 106]
Fehlerhafte Koeffizienten (Potenzen):  [18, 22]
Fehlerhafte Indizes:  [3, 7]


In [25]:
# 3. Fehlerkorrektur

# 1. Fehler-Auswerter-Polynom Ω(x) (Omega) berechnen
syndromes_poly = syndromes[:]

num_errors = len(locator_poly) - 1

# Polynom-Multiplikation durchführen UND auf den richtigen Grad kürzen
omega_full = gf_poly_multiply(syndromes_poly, locator_poly)
omega_poly = omega_full[:num_errors]

print("Fehler-Auswerter-Polynom Ω(x):", omega_poly)

# 2. Ableitung von Λ'(x) berechnen
print("Λ(x):", locator_poly_lf)
lambda_derivative = []
n = len(locator_poly_lf) - 1
for i in range(1, len(locator_poly_lf)):
    coef = gf_multiply(i, locator_poly_lf[i])
    lambda_derivative.append(coef)

print("Ableitung Λ'(x):", lambda_derivative)

# ----------------

# 3. Fehlerwerte (Magnituden) für jeden Index berechnen
corrected_codeword = damaged.copy()
codeword_length = len(damaged)

for index in error_indices:
    position = index
    field_position = codeword_length - 1 - position  # Chien-Suchposition
    x_inv = exp_table[255 - field_position]

    # 1. Ω(x⁻¹) und Λ′(x⁻¹) auswerten im GF(2⁸)
    omega_val = poly_eval(omega_poly, x_inv, exp_table, log_table)
    lambda_prime_val = poly_eval(lambda_derivative, x_inv, exp_table, log_table)

    # 2. Inverses von Λ′(x⁻¹) berechnen
    lambda_prime_inv = exp_table[(255 - log_table[lambda_prime_val]) % 255]

    # 3. Fehlerwert berechnen
    magnitude = gf_multiply(omega_val, lambda_prime_inv)

    # 4. Fehler korrigieren
    before = corrected_codeword[index]
    corrected_codeword[index] ^= magnitude
    after = corrected_codeword[index]

    # 5. Debug-Ausgabe
    print(f"\nFehler an Byte-Index {index}:")
    print(f"  → Feldposition (für x⁻¹): {field_position}")
    print(f"  → x_inv (α^−pos): {x_inv}")
    print(f"  → Ω(x_inv): {omega_val}")
    print(f"  → Λ'(x_inv): {lambda_prime_val}")
    print(f"  → Fehlerwert = {magnitude}, XOR: {before} ^ {magnitude} = {after}")

# Überprüfung:
print("\nOriginales, korrektes Codewort: ", correct)
print("Beschädigtes Codewort:        : ", damaged)
print("Wiederhergestelltes Codewort  : ", corrected_codeword)

if corrected_codeword == correct:
    print("\nKorrektur ERFOLGREICH!")
else:
    print("\nKorrektur FEHLGESCHLAGEN!")

Fehler-Auswerter-Polynom Ω(x): [125, 64]
Λ(x): [1, 199, 106]
Ableitung Λ'(x): [199, 212]

Fehler an Byte-Index 3:
  → Feldposition (für x⁻¹): 22
  → x_inv (α^−pos): 243
  → Ω(x_inv): 5
  → Λ'(x_inv): 13
  → Fehlerwert = 56, XOR: 34 ^ 56 = 26

Fehler an Byte-Index 7:
  → Feldposition (für x⁻¹): 18
  → x_inv (α^−pos): 139
  → Ω(x_inv): 100
  → Λ'(x_inv): 197
  → Fehlerwert = 101, XOR: 111 ^ 101 = 10

Originales, korrektes Codewort:  [69, 76, 68, 65, 82, 236, 17, 236, 17, 236, 17, 236, 17, 29, 245, 55, 210, 123, 88, 72, 194, 191, 232, 62, 160, 51]
Beschädigtes Codewort:        :  [69, 76, 68, 34, 82, 236, 17, 111, 17, 236, 17, 236, 17, 29, 245, 55, 210, 123, 88, 72, 194, 191, 232, 62, 160, 51]
Wiederhergestelltes Codewort  :  [69, 76, 68, 26, 82, 236, 17, 10, 17, 236, 17, 236, 17, 29, 245, 55, 210, 123, 88, 72, 194, 191, 232, 62, 160, 51]

Korrektur FEHLGESCHLAGEN!
